In [1]:
a <- 100*3

In [3]:
getwd()

[1] "f:/kiran/ml model on agric/RNA_seq/analysis"

In [5]:
library(dplyr)
library(tidyverse)
library(GEOquery)

ERROR: Error in library(tidyverse): there is no package called 'tidyverse'
